In [2]:
#Importing files

import cv2
import numpy as np
import pickle

#Creating grid
def build_squares(img):
	x, y, w, h = 420, 140, 10, 10    #x,y co-ord; #w,h wid ht
	d = 10        #10 rows of boxes
	imgCrop = None
	crop = None
	for i in range(10):    #for 10 rows    
		for j in range(5):    #for 5 column
			if np.any(imgCrop == None):     #for 1st row 1st block is print
				imgCrop = img[y:y+h, x:x+w]
			else:                           #For 1st row rest 4 block is print
				imgCrop = np.hstack((imgCrop, img[y:y+h, x:x+w]))   
			#print(imgCrop.shape)
			cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 1)  #cv2.rectangle(image, start_point, end_point, color, thickness)
			x+=w+d   #to define postion among horizontal boxs
		if np.any(crop == None):    #for 1st col block is print
			crop = imgCrop
		else:                   #for 1st col rest 9 block is print
			crop = np.vstack((crop, imgCrop)) 
		imgCrop = None
		x = 420   #x value has been changed to need to define again the original value so ot dont get overlapped 
		y+=h+d     #to define postion among vertical boxs   #
	return crop

def get_hand_hist():
	cam = cv2.VideoCapture(1)  #for starting the camera and recording the gesture
	if cam.read()[0]==False:
		cam = cv2.VideoCapture(0)
	x, y, w, h = 300, 100, 300, 300     #thresh box
	flagPressedC, flagPressedS = False, False  # by pressing C thresh is getting opened and by S it is getting killed
	imgCrop = None
	while True:
		img = cam.read()[1]   #taking ip from cam
		img = cv2.flip(img, 1)  #cv.2flip(src,flipcode,destination)  #src, dest = imag; 1 = flip code and associated with y axis
		img = cv2.resize(img, (640, 480))  #shrinking the image grid
		hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)      #convert RGB/BGR to HSV (hue saturation value)
		
		keypress = cv2.waitKey(1)   #if 0 the it will be still and if 1 it will capture live image
		if keypress == ord('c'):   #check to Ascii Value of C		
			hsvCrop = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2HSV)
			flagPressedC = True
			hist = cv2.calcHist([hsvCrop], [0, 1], None, [180, 256], [0, 180, 0, 256])  #cv.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
			cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
		elif keypress == ord('s'):  #check to Ascii Value of S
			flagPressedS = True	
			break
		if flagPressedC:	
			dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)
			dst1 = dst.copy()
			disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))# shape=cv2.MORPH_ELLIPSE, ksize=(kernel_size, kernel_size
			cv2.filter2D(dst,-1,disc,dst) #OpenCV provides a function cv.filter2D() to convolve a kernel with an image
			blur = cv2.GaussianBlur(dst, (11,11), 0) #cv::GaussianBlur (InputArray src, OutputArray dst, Size ksize, double sigmaX, double sigmaY=0, int borderType=BORDER_DEFAULT) Blurs an image using a Gaussian filter
			blur = cv2.medianBlur(blur, 15) #cv::medianBlur (InputArray src, OutputArray dst, int ksize) Blurs an image using the median filter
			ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
			thresh = cv2.merge((thresh,thresh,thresh)) #takes single channel images and combines them to make a multi-channel image
			#cv2.imshow("res", res)
			cv2.imshow("Thresh", thresh) #window name and image
		if not flagPressedS:
			imgCrop = build_squares(img) 
		#cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
		cv2.imshow("Set hand histogram", img)#window name and image
	cam.release() #closing camera
	cv2.destroyAllWindows()
	#Pickle serializes objects so they can be saved to a file, and loaded in a program again later on.
	with open("hist", "wb") as f:
		pickle.dump(hist, f)


get_hand_hist()  #end of the function
